In [1]:
import pandas as pd

In [6]:
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df = df.dropna()

In [8]:
df.shape

(18285, 5)

In [9]:
X = df.drop('label',axis=1)

In [10]:
y = df['label']

In [11]:
X.shape

(18285, 4)

In [12]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [14]:
#Vocabulary Size
voc_size = 5000

In [15]:
messages = X.copy()
messages.reset_index(inplace=True)


In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
#Data Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

One Hot Representation

In [21]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
onehot_repr

[[667, 2885, 3503, 4231, 810, 4349, 3330, 885, 1053, 825],
 [1966, 4596, 2803, 1474, 772, 2419, 4051],
 [870, 3843, 1352, 1923],
 [2342, 3624, 2791, 1421, 704, 4314],
 [88, 772, 1919, 832, 554, 3594, 772, 50, 96, 1818],
 [898,
  3475,
  3205,
  980,
  4503,
  4710,
  1907,
  4183,
  2902,
  18,
  3976,
  3465,
  4941,
  4713,
  4051],
 [941, 2836, 849, 430, 1880, 2396, 2076, 630, 2952, 893, 4766],
 [885, 2880, 2734, 2409, 862, 3447, 4710, 4865, 2952, 893, 4766],
 [2594, 1617, 3740, 4081, 4927, 619, 4058, 3370, 4710, 3660],
 [1815, 770, 1880, 3635, 4779, 760, 4523, 217],
 [2595, 3363, 3326, 1343, 2731, 2332, 2309, 1039, 1282, 3914, 3577],
 [1421, 1229, 810, 619, 4710, 862],
 [1111, 3592, 2823, 3828, 2284, 570, 4387, 4300, 257],
 [1010, 4464, 4405, 1241, 4310, 724, 4956, 2952, 893, 4766],
 [4765, 1344, 3168, 4582, 1603, 2952, 893, 4766],
 [1313, 1269, 4461, 1593, 3691, 292, 57, 3171, 4346, 4227],
 [4768, 626, 4596],
 [4010, 165, 2597, 399, 4710, 4674, 3928, 4051],
 [2711, 1608, 2803, 198

Embedding Representation

In [22]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  885 1053  825]
 [   0    0    0 ...  772 2419 4051]
 [   0    0    0 ... 3843 1352 1923]
 ...
 [   0    0    0 ... 2952  893 4766]
 [   0    0    0 ... 1252  126 3061]
 [   0    0    0 ... 4603  278 3961]]


In [23]:
print(len(embedded_docs))
print(embedded_docs[0])

18285
[   0    0    0    0    0    0    0    0    0    0  667 2885 3503 4231
  810 4349 3330  885 1053  825]


In [38]:
## Creating a model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100)) #We use 1 LSTM layer with 100 neurons
model.add(Dense(1,activation='sigmoid')) #Dense layer helps in finding the classification output
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [39]:
len(embedded_docs),y.shape

(18285, (18285,))

In [40]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [41]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [42]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

Model Training

In [43]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 40ms/step - loss: 0.3325 - accuracy: 0.8438 - val_loss: 0.2005 - val_accuracy: 0.9157
Epoch 2/10
192/192 [==============================] - 2s 12ms/step - loss: 0.1359 - accuracy: 0.9482 - val_loss: 0.2204 - val_accuracy: 0.9022
Epoch 3/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0977 - accuracy: 0.9646 - val_loss: 0.2433 - val_accuracy: 0.9065
Epoch 4/10
192/192 [==============================] - 1s 6ms/step - loss: 0.0693 - accuracy: 0.9767 - val_loss: 0.3082 - val_accuracy: 0.8999
Epoch 5/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0468 - accuracy: 0.9837 - val_loss: 0.3008 - val_accuracy: 0.9148
Epoch 6/10
192/192 [==============================] - 2s 8ms/step - loss: 0.0281 - accuracy: 0.9908 - val_loss: 0.3975 - val_accuracy: 0.9143
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0163 - accuracy: 0.9949 - val_loss: 0.4882 - val_accuracy: 0.9102
Epo

Adding dropout

In [30]:
from tensorflow.keras.layers import Dropout
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))

Performance Metrics and Accuracy

In [44]:
y_pred = model.predict(X_test)
y_pred_class = np.argmax(y_pred, axis=1)

189/189 [==============================] - 1s 2ms/step


In [45]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_class)

array([[3419,    0],
       [2616,    0]])